In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import timm
import os

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [2]:
CHECKPOINT_PATH = r"C:\Users\khotc\Downloads\skin\checkpoints\Swin_MC_best_model.pth"
TEST_CSV = r"C:\Users\khotc\Downloads\skin\METADATA\test_split.csv"


In [3]:
df = pd.read_csv(TEST_CSV)

# Main_class वापरून accuracy
image_paths = df['image_path'].tolist()  # CSV मध्ये image_path column असावा
labels = df['Main_class'].tolist()

unique_classes = sorted(df['Main_class'].unique())
class_to_idx = {cls: idx for idx, cls in enumerate(unique_classes)}
idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}

numeric_labels = [class_to_idx[l] for l in labels]
num_classes = len(unique_classes)



KeyError: 'image_path'

In [4]:
import torch
import timm
import torch.nn as nn

# Model definition (साचाच वापर करा जसा train करताना केला होता)
class SwinClassifier(nn.Module):
    def __init__(self, num_classes: int, model_name: str = "swin_base_patch4_window12_384"):
        super().__init__()
        self.backbone = timm.create_model(
            model_name,
            pretrained=False,
            num_classes=num_classes,
            global_pool="avg",
            img_size=512
        )
    def forward(self, x):
        return self.backbone(x)

CHECKPOINT_PATH = r'C:\Users\khotc\Downloads\skin\checkpoints\Swin_MC_best_model.pth'
NUM_CLASSES = 8  # तुझ्या trained model च्या output classes (जे train करताना used होते)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model instantiate
model = SwinClassifier(num_classes=NUM_CLASSES).to(device)

# Try loading checkpoint
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

# Simple key check
print("Checkpoint keys:", checkpoint.keys() if isinstance(checkpoint, dict) else "Not a dict")

# Load state_dict safely
try:
    model.load_state_dict(checkpoint, strict=True)  # strict=True म्हणजे mismatch alert
    print("✅ Model loaded correctly!")
except Exception as e:
    print("❌ Error loading model:", e)

# Optional: Minimal forward pass with dummy tensor
dummy_input = torch.randn(1, 3, 512, 512).to(device)
try:
    out = model(dummy_input)
    print("✅ Forward pass successful! Output shape:", out.shape)
except Exception as e:
    print("❌ Forward pass failed:", e)


Checkpoint keys: odict_keys(['patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'layers.0.blocks.0.norm1.weight', 'layers.0.blocks.0.norm1.bias', 'layers.0.blocks.0.attn.relative_position_bias_table', 'layers.0.blocks.0.attn.qkv.weight', 'layers.0.blocks.0.attn.qkv.bias', 'layers.0.blocks.0.attn.proj.weight', 'layers.0.blocks.0.attn.proj.bias', 'layers.0.blocks.0.norm2.weight', 'layers.0.blocks.0.norm2.bias', 'layers.0.blocks.0.mlp.fc1.weight', 'layers.0.blocks.0.mlp.fc1.bias', 'layers.0.blocks.0.mlp.fc2.weight', 'layers.0.blocks.0.mlp.fc2.bias', 'layers.0.blocks.1.norm1.weight', 'layers.0.blocks.1.norm1.bias', 'layers.0.blocks.1.attn.relative_position_bias_table', 'layers.0.blocks.1.attn.qkv.weight', 'layers.0.blocks.1.attn.qkv.bias', 'layers.0.blocks.1.attn.proj.weight', 'layers.0.blocks.1.attn.proj.bias', 'layers.0.blocks.1.norm2.weight', 'layers.0.blocks.1.norm2.bias', 'layers.0.blocks.1.mlp.fc1.weight', 'layers.0.blocks.1.mlp.fc

In [6]:
import torch

# Model define kara
model = SwinClassifier(num_classes=7)  # tula class count change karaycha asel tar change kara

# Checkpoint load kara
checkpoint_path = 'C:\Users\khotc\Downloads\skin\checkpoints\Swin_MC_best_model.pth'  # tuzya checkpoint cha path
checkpoint = torch.load(checkpoint_path, map_location='cpu')  # GPU nahi tar cpu use kara

# Keys la modify kara ('backbone.' prefix add kara)
new_state_dict = {}
for k, v in checkpoint.items():
    new_state_dict['backbone.' + k] = v

# Model madhye load kara
model.load_state_dict(new_state_dict)

# Evaluation mode set kara
model.eval()

print("✅ Model loaded successfully with modified keys!")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (45618837.py, line 7)